In [3]:
%load_ext autoreload
%autoreload 2
import json
import cv2
import os
from ultralytics import YOLO
from YOLOv8_predict import extract_detections_pt
from utils import parse_coco_dataset, crop_box, draw_objects, calculate_iou

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# initialize paths
folder_with_images = '/data/datasets/model_validation/val_yolo/images'
coco_annotations_file = '/data/datasets/model_validation/coco/annotations/instances_default.json'
visualized_images = 'visualized_images'
yolo_model_name = "yolov8n"

In [3]:
#  functions
def draw_and_save_coco(coco_annotations, result_folder):
    os.makedirs(result_folder, exist_ok=True)
    for image_filepath, objects in coco_annotations.items():
        image = cv2.imread(image_filepath)
        if image is None:
            print(f"Cannot read image. Check path {image_filepath}")
            continue
        image = draw_objects(image, objects)
        result_filepath = os.path.join(result_folder, os.path.split(image_filepath)[-1])
        cv2.imwrite(result_filepath, image)

def run_image_with_yolo(image, model):
    detections = []
    predicted_data = model(image)
    for pred in predicted_data:
        dets = extract_detections_pt(pred)
        for det in dets:
            label = det['label']
            score = det['score']
            box = crop_box(det['ltrb'], image.shape[:2])
            detections.append({'label': label, 'ltrb': box, 'score': score})
    return detections

def process_images(image_filepaths, model, annotations={}, result_folder=None):
    if result_folder: os.makedirs(result_folder, exist_ok=True)
    images_annotations_detections = {}
    for image_filepath in image_filepaths:
        # read image
        image = cv2.imread(image_filepath)
        if image is None:
            print(f"Cannot read image. Check path {image_filepath}")
            continue
        # get annotated objects if annotations provided
        annotated_objects = annotations.get(image_filepath, [])
        # get detections
        detections = run_image_with_yolo(image, model)
        # draw objects and save an image if required
        if result_folder:
            result_filepath = os.path.join(result_folder, os.path.split(image_filepath)[-1])
            image = draw_objects(image, annotated_objects)
            image = draw_objects(image, detections)
            cv2.imwrite(result_filepath, image)
        #  keep results for next evaluation
        images_annotations_detections[image_filepath] = {
            "annotated_objects": annotated_objects,
            "detections": detections
        }
    return images_annotations_detections

In [4]:
coco_annotations = parse_coco_dataset(folder_with_images, coco_annotations_file)
model = YOLO(yolo_model_name)
images_annotations_detections = process_images(coco_annotations.keys(), model, coco_annotations, visualized_images)


0: 384x640 3 persons, 1 horse, 85.6ms
Speed: 0.8ms preprocess, 85.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 3.8ms
Speed: 1.0ms preprocess, 3.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 3.8ms
Speed: 1.0ms preprocess, 3.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 3.8ms
Speed: 0.8ms preprocess, 3.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 1 cow, 3.8ms
Speed: 1.0ms preprocess, 3.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 3.8ms
Speed: 1.0ms preprocess, 3.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 1 cow, 3.9ms
Speed: 1.1ms preprocess, 3.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 3.8ms
Speed: 0.9ms preprocess, 3.8ms inference, 0.6ms postprocess per image a

In [5]:
def compute_ious(img_filepath, data):
    results = []
    ground_truth_data = data['annotated_objects']
    predicted_data = data['detections']
    for gt_obj in ground_truth_data:
            gt_label, gt_box = gt_obj['label'], gt_obj['ltrb']
            for detection in predicted_data:
                det_label, det_box, det_score = detection['label'], detection['ltrb'], detection['score']
                if gt_label == det_label:
                    iou = calculate_iou(gt_box, det_box)
                    results.append({
                        'img_filepath': img_filepath, 
                        'label': gt_label, 
                        'gt_box': gt_box,
                        'det_box': det_box,
                        'iou': iou})
    return results
img_filepaths = list(coco_annotations.keys())
img_filepath  = img_filepaths[0]
results = compute_ious(img_filepath, images_annotations_detections[img_filepath])
print(results)

[{'img_filepath': '/data/datasets/model_validation/val_yolo/images/frame_000000.PNG', 'label': 'person', 'gt_box': [906.5, 372.1, 967.6, 528.74], 'det_box': [907, 372, 965, 520], 'iou': 0.895754860019898}, {'img_filepath': '/data/datasets/model_validation/val_yolo/images/frame_000000.PNG', 'label': 'person', 'gt_box': [906.5, 372.1, 967.6, 528.74], 'det_box': [928, 475, 990, 617], 'iou': 0.13098765280120156}, {'img_filepath': '/data/datasets/model_validation/val_yolo/images/frame_000000.PNG', 'label': 'person', 'gt_box': [906.5, 372.1, 967.6, 528.74], 'det_box': [162, 421, 206, 520], 'iou': 0.0}, {'img_filepath': '/data/datasets/model_validation/val_yolo/images/frame_000000.PNG', 'label': 'person', 'gt_box': [158.42, 412.53, 213.51, 527.01], 'det_box': [907, 372, 965, 520], 'iou': 0.0}, {'img_filepath': '/data/datasets/model_validation/val_yolo/images/frame_000000.PNG', 'label': 'person', 'gt_box': [158.42, 412.53, 213.51, 527.01], 'det_box': [928, 475, 990, 617], 'iou': 0.0}, {'img_fi

In [6]:
def get_tp_fn_fp(ann_obj, detections, iou_threshold=0.7, conf_threshold=0.8):
    def is_in(box, label, lst, iou_threshold, conf_threshold):
        for obj in lst:
            if not label == obj['label']: continue
            if obj.get('score', 1.0) <= conf_threshold: continue
            if calculate_iou(box, obj['ltrb']) < iou_threshold: continue
            return True
        return False
    
    tp, fn = 0, 0
    for gt in ann_obj:
        if is_in(gt['ltrb'], gt['label'], detections, iou_threshold, conf_threshold): tp += 1
        else: fn += 1
    fp = 0
    for pred in detections:
        if is_in(pred['ltrb'], pred['label'], ann_obj, iou_threshold, conf_threshold): continue
        else: fp += 1   
    return tp, fn, fp

In [11]:
def evaluate(images_annotations_detections, iou_threshold=0.7, conf_threshold=0.8):
    tps, fns, fps = 0, 0, 0
    for _, info in images_annotations_detections.items():
        annotated_obj = info["annotated_objects"]
        detections = info['detections']
        tp, fn, fp = get_tp_fn_fp(annotated_obj, detections, iou_threshold, conf_threshold)
        tps += tp
        fns += fn
        fps += fp
    # pre, rec, f1s
    precision = tps/float(tps+fps) if tps+fps>0 else 0.0
    recall = tps/float(tps+fns) if tps+fns>0 else 0.0
    f1s = (2*precision*recall)/(precision+recall) if precision+recall>0 else 0.0
    return tps, fns, fps, precision, recall, f1s

img_filepaths = list(coco_annotations.keys())
image_filepath  = img_filepaths[0]

evaluate(images_annotations_detections, iou_threshold=0.7, conf_threshold=0.8)

(469,
 40263,
 5613,
 0.07711279184478789,
 0.01151428852008249,
 0.020036741145811082)